# Init

In [164]:
# imports
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

---
#### Load data

In [165]:
num_video = 0
num_endpoint = 0
num_req_descriptions = 0
num_server = 0

cache_capacity = 0
video_size = []

latency = defaultdict(lambda: defaultdict(int))                         # [endpoint][cache/datacenter] = latency
reqs = defaultdict(lambda: defaultdict(int))        # [endpoint][video] = num reqs

In [166]:
# dataset = "dataset/videos_worth_spreading.in"
# dataset = "dataset/me_at_the_zoo.in"
# dataset = "dataset/custom.in"
dataset = "dataset/minimal.in"

status = 0
endpoint_index = 0
num_connected_cache = 0
with open(dataset, "r") as f:
    for line_content in f:
        line = line_content.split()

        if status ==0:                                  # get counters
            num_video = int(line[0])
            num_endpoint = int(line[1])
            num_req_descriptions = int(line[2])
            num_server = int(line[3])
            cache_capacity = int(line[4])
            status = 1

        elif status == 1:                               # get video dims
            for size in line:
                video_size.append(int(size))
            status = 2

        elif status == 2:                               # get datacenter latency and connected cache number
            data_center_latency = int(line[0])
            latency[endpoint_index][num_server] = data_center_latency
            
            num_connected_cache = int(line[1])
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
            else:
                status = 3
        
        elif status == 3:                                  # get cache latency
            cache_index = int(line[0])
            cache_latency = int(line[1])
            latency[endpoint_index][cache_index] = cache_latency
            
            num_connected_cache = num_connected_cache - 1
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
                else:
                    status = 2
        
        elif status == 4:                                   # take num requests
            video_index = int(line[0])
            endpoint_index = int(line[1])
            num_reqs = int(line[2])
            reqs[endpoint_index][video_index] = num_reqs                         

In [167]:
print(f"num video: {num_video}, num endpoints {num_endpoint}, req descriptions {num_req_descriptions}, num cache {num_server}, dim {cache_capacity}")
print(f"video sized: {video_size}")
print(f"latencies: {latency}")
print(f"reqs: {reqs}")

num video: 2, num endpoints 1, req descriptions 2, num cache 1, dim 100
video sized: [50, 50]
latencies: defaultdict(<function <lambda> at 0x7f978c785b20>, {0: defaultdict(<class 'int'>, {1: 1000, 0: 100})})
reqs: defaultdict(<function <lambda> at 0x7f9795de9760>, {0: defaultdict(<class 'int'>, {0: 1000, 1: 500})})


---
# Math model

In [168]:
endpoint_index = range(num_endpoint)
server_index = range(num_server + 1) # I've modelled datacenter as last server
video_index = range(num_video)

In [169]:
model = gp.Model("YoutubeCache")

# decision vars
x = model.addVars(endpoint_index, server_index, video_index, vtype=gp.GRB.BINARY, name="x")
y = model.addVars(server_index, video_index, vtype=gp.GRB.BINARY, name="y")

# objective function
obj = gp.quicksum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
model.setObjective(obj, GRB.MINIMIZE)
# model.setObjective(gp.quicksum(y[s,v] for s in server_index for v in video_index), GRB.MAXIMIZE)


# constraints
constr = (gp.quicksum(x[e,s,v] for e in endpoint_index)  <= num_endpoint*y[s,v] for s in server_index for v in video_index )
model.addConstrs(constr, name="if video v available on server s")

constr = ( gp.quicksum( x[e,s,v] for s in server_index ) == (1 if reqs[e][v] else 0) for e in endpoint_index for v in video_index ) # datacenter excluded 
# constr = ( gp.quicksum( x[e,s,v] for s in latency[e] ) == (1 if reqs[e][v] else 0) for e in endpoint_index for v in video_index ) # latency[e] directly check existring connections
model.addConstrs(constr, name="every request must be satisfied")

constr = ( gp.quicksum(video_size[v] * y[s,v] for v in video_index) <= cache_capacity for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="cache capacity")


constr = ( y[num_server,v] == 1 for v in video_index ) # cache servers are from 0 to s-1, s index (num_server) is for datacenter
model.addConstrs(constr, name="Datacenter have all videos")

constr = ( gp.quicksum( x[e,s,v] for v in video_index ) <= (num_video*latency[e][s]) for e in endpoint_index for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="video v must be available on server s to be selected")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>}

In [170]:
# Optimize the model
model.optimize()

model.display()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 8 columns and 18 nonzeros
Model fingerprint: 0xc33683a7
Variable types: 0 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e+04, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 1500000.0000
Presolve removed 10 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 150000 1.5e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+05, best bound 1.500000000000e+05, gap 0.0000%
Minimize
  100000.0 x[0,0,0] + 50000

/tmp/ipykernel_6503/1976515759.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()


In [171]:

# results
if model.status == gp.GRB.OPTIMAL:
    print("\nOptimization successful!")
    print("Optimal X values:")
    for e in endpoint_index:
        for s in server_index:
            for v in video_index:
                print(f"X[{e},{s},{v}] * {latency[e][s]} = {x[e,s,v]}")
    print("\nOptimal Y values:")
    for s in server_index:
        for v in video_index:
            print(f"Y[{s},{v}] = {y[s,v]}")


    print(f"\nOptimal objective value: {model.objVal}")
elif model.status == gp.GRB.INFEASIBLE:
    print("Model is infeasible.")
elif model.status == gp.GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization ended with status {model.status}")


Optimization successful!
Optimal X values:
X[0,0,0] * 100 = <gurobi.Var x[0,0,0] (value 1.0)>
X[0,0,1] * 100 = <gurobi.Var x[0,0,1] (value 1.0)>
X[0,1,0] * 1000 = <gurobi.Var x[0,1,0] (value 0.0)>
X[0,1,1] * 1000 = <gurobi.Var x[0,1,1] (value 0.0)>

Optimal Y values:
Y[0,0] = <gurobi.Var y[0,0] (value 1.0)>
Y[0,1] = <gurobi.Var y[0,1] (value 1.0)>
Y[1,0] = <gurobi.Var y[1,0] (value 1.0)>
Y[1,1] = <gurobi.Var y[1,1] (value 1.0)>

Optimal objective value: 150000.0
